In [20]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, Embedding, Input, Reshape, concatenate
from keras import metrics
from keras.optimizers import Adam, rmsprop
from keras.callbacks import TensorBoard, ReduceLROnPlateau
from time import time
from IPython.display import display

In [2]:
experiment_name='mlhw-mlp-with-preprocess'

## Load Dataset

In [3]:
data_val = pd.read_csv('dataset/train_values.csv')
data_labels = pd.read_csv('dataset/train_labels.csv')

# Merge values and label
data = data_val.merge(data_labels, on='patient_id')
data = data.drop(columns='patient_id')

# Convert string values to integers
data["thal"] = data["thal"].astype('category')
data["thal"] = data["thal"].cat.codes

# Split dataset
train = data.sample(frac=0.8)
validate = data.sample(frac=0.2)

xsize = train.shape[1] - 1
ysize = 1
label_column = 'heart_disease_present'
cols = list(train.columns)
cols.remove(label_column)



In [4]:
display(train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
41,1,1,135,3,0,0,2,252,0.0,0,63,172,0,0
169,2,1,110,1,0,0,2,211,1.8,1,64,144,1,0
64,1,1,110,4,0,0,2,254,0.0,0,50,159,0,0
147,2,1,134,1,2,0,0,234,2.6,1,61,145,0,1
137,2,1,150,4,0,0,0,244,1.4,0,62,154,1,1


## Preprocessing

### Data Whitening
This will normalize our values so higher order values won't dominate other values

In [5]:
train_mean = train[cols].mean(axis=0)
train_std = train[cols].std(axis=0)
train[cols] = (train[cols] - train_mean) / train_std
validate[cols] = (validate[cols] - train_mean) / train_std

display(train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
41,-0.905871,-0.611182,0.241335,-0.144988,-0.752141,-0.41176,0.993080,0.084638,-0.936866,-1.387556,0.870568,1.013838,-0.693665,0
169,0.788226,-0.611182,-1.248897,-2.232814,-0.752141,-0.41176,0.993080,-0.771016,0.786595,0.715687,0.977440,-0.239702,1.431607,0
64,-0.905871,-0.611182,-1.248897,0.898925,-0.752141,-0.41176,0.993080,0.126377,-0.936866,-1.387556,-0.518778,0.431837,-0.693665,0
147,0.788226,-0.611182,0.181726,-2.232814,1.350933,-0.41176,-1.006969,-0.291015,1.552578,0.715687,0.656822,-0.194933,-0.693665,1
137,0.788226,-0.611182,1.135474,0.898925,-0.752141,-0.41176,-1.006969,-0.082319,0.403604,-1.387556,0.763695,0.207991,1.431607,1


In [14]:
# TODO: For Categorical Embeddings
cat_cols = [
    'thal',
    'chest_pain_type',
    'num_major_vessels',
    'fasting_blood_sugar_gt_120_mg_per_dl',
    'resting_ekg_results',
    'sex',
    'exercise_induced_angina'
]
cont_cols = [
    'slope_of_peak_exercise_st_segment',
    'resting_blood_pressure',
    'serum_cholesterol_mg_per_dl'
    'oldpeak_eq_st_depression',
    'age',
    'max_heart_rate_achieved',
]


## Define Model 

In [21]:
inputs = []
for cat_col in cat_cols :
     
    no_of_unique_cat  = train[cat_col].nunique()
    embedding_size = min(np.ceil((no_of_unique_cat)/2), 50 )
    embedding_size = int(embedding_size)
    vocab  = no_of_unique_cat + 1
    
    emb_input = Input(shape=(1,))
    x = Embedding(vocab ,embedding_size, input_length = 1)(emb_input)
    emb_output = Reshape(target_shape=(embedding_size, ))(x)
    inputs.append(x)

cont_input = Input(shape=(len(cont_cols),))
inputs.append(cont_input)

x = concatenate(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(256)(x)

x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(128)(x)

x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(64)(x)

x = Dense(ysize)(x)
model_output = Activation('sigmoid')(x)

model = Model(
    inputs=inputs, 
    outputs=[model_output]
)

initial_lr = 0.001

opt_name = 'rmsprop'
opt = rmsprop(lr=initial_lr, decay=1e-6)

#opt_name='adam'
#opt = Adam(lr=initial_lr)


model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=[metrics.binary_accuracy]
)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 1, 2), (None, 1, 2), (None, 1, 2), (None, 1, 1), (None, 1, 2), (None, 1, 1), (None, 1, 1), (None, 5)]

## Training

In [8]:
epochs = 500
batch_size = 32
exp_stamp = '{}-ilr={}-opt={}-{}'.format(experiment_name, initial_lr, opt_name, time())

tb_cb = TensorBoard(
    batch_size=batch_size,
    log_dir="logs/{}".format(exp_stamp),
    histogram_freq=0,
    write_graph=True,
    write_images=True,
    update_freq='batch'
)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [tb_cb, lr_reducer]

model.fit(
    train[cols].values,
    train[label_column].values,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols].values,validate[label_column].values),
    callbacks=callbacks
)

# Save model and weights
save_dir=os.path.join(os.getcwd(), 'saved_models')
model_name= '{}.h5'.format(exp_stamp)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Train on 144 samples, validate on 36 samples
Epoch 1/500
144/144 [==============================] - 1s 5ms/step - loss: 0.6696 - binary_accuracy: 0.6319 - val_loss: 0.2329 - val_binary_accuracy: 0.9167
Epoch 2/500
144/144 [==============================] - 0s 320us/step - loss: 0.3286 - binary_accuracy: 0.8819 - val_loss: 0.2170 - val_binary_accuracy: 0.8889
Epoch 3/500
144/144 [==============================] - 0s 329us/step - loss: 0.2937 - binary_accuracy: 0.8958 - val_loss: 0.1689 - val_binary_accuracy: 0.9167
Epoch 4/500
144/144 [==============================] - 0s 355us/step - loss: 0.2437 - binary_accuracy: 0.9097 - val_loss: 0.2168 - val_binary_accuracy: 0.8889
Epoch 5/500
144/144 [==============================] - 0s 324us/step - loss: 0.2340 - binary_accuracy: 0.9167 - val_loss: 0.1447 - val_binary_accuracy: 0.9444
Epoch 6/500
144/144 [==============================] - 0s 320us/step - loss: 0.1956 - binary_accuracy: 0.9306 - val_loss: 0.1958 - val_binary_accuracy: 0.8611
Epo

Epoch 52/500
144/144 [==============================] - 0s 317us/step - loss: 0.0802 - binary_accuracy: 0.9792 - val_loss: 0.1612 - val_binary_accuracy: 0.9444
Epoch 53/500
144/144 [==============================] - 0s 280us/step - loss: 0.1111 - binary_accuracy: 0.9514 - val_loss: 0.1611 - val_binary_accuracy: 0.9444
Epoch 54/500
144/144 [==============================] - 0s 267us/step - loss: 0.1219 - binary_accuracy: 0.9653 - val_loss: 0.1609 - val_binary_accuracy: 0.9444
Epoch 55/500
144/144 [==============================] - 0s 268us/step - loss: 0.0760 - binary_accuracy: 0.9861 - val_loss: 0.1609 - val_binary_accuracy: 0.9444
Epoch 56/500
144/144 [==============================] - 0s 316us/step - loss: 0.0606 - binary_accuracy: 0.9861 - val_loss: 0.1614 - val_binary_accuracy: 0.9444
Epoch 57/500
144/144 [==============================] - 0s 259us/step - loss: 0.0676 - binary_accuracy: 0.9861 - val_loss: 0.1615 - val_binary_accuracy: 0.9444
Epoch 58/500
144/144 [==================

Epoch 154/500
144/144 [==============================] - 0s 396us/step - loss: 0.0952 - binary_accuracy: 0.9722 - val_loss: 0.1631 - val_binary_accuracy: 0.9444
Epoch 155/500
144/144 [==============================] - 0s 317us/step - loss: 0.1112 - binary_accuracy: 0.9583 - val_loss: 0.1622 - val_binary_accuracy: 0.9444
Epoch 156/500
144/144 [==============================] - 0s 321us/step - loss: 0.1047 - binary_accuracy: 0.9583 - val_loss: 0.1624 - val_binary_accuracy: 0.9444
Epoch 157/500
144/144 [==============================] - 0s 306us/step - loss: 0.1235 - binary_accuracy: 0.9653 - val_loss: 0.1628 - val_binary_accuracy: 0.9444
Epoch 158/500
144/144 [==============================] - 0s 343us/step - loss: 0.1078 - binary_accuracy: 0.9653 - val_loss: 0.1633 - val_binary_accuracy: 0.9444
Epoch 159/500
144/144 [==============================] - 0s 310us/step - loss: 0.0643 - binary_accuracy: 0.9931 - val_loss: 0.1633 - val_binary_accuracy: 0.9444
Epoch 160/500
144/144 [===========

Epoch 205/500
144/144 [==============================] - 0s 291us/step - loss: 0.0708 - binary_accuracy: 0.9792 - val_loss: 0.1622 - val_binary_accuracy: 0.9444
Epoch 206/500
144/144 [==============================] - 0s 313us/step - loss: 0.0763 - binary_accuracy: 0.9792 - val_loss: 0.1622 - val_binary_accuracy: 0.9444
Epoch 207/500
144/144 [==============================] - 0s 323us/step - loss: 0.0950 - binary_accuracy: 0.9792 - val_loss: 0.1632 - val_binary_accuracy: 0.9444
Epoch 208/500
144/144 [==============================] - 0s 318us/step - loss: 0.0822 - binary_accuracy: 0.9792 - val_loss: 0.1631 - val_binary_accuracy: 0.9444
Epoch 209/500
144/144 [==============================] - 0s 319us/step - loss: 0.0733 - binary_accuracy: 0.9792 - val_loss: 0.1629 - val_binary_accuracy: 0.9444
Epoch 210/500
144/144 [==============================] - 0s 296us/step - loss: 0.0859 - binary_accuracy: 0.9722 - val_loss: 0.1628 - val_binary_accuracy: 0.9444
Epoch 211/500
144/144 [===========

Epoch 256/500
144/144 [==============================] - 0s 356us/step - loss: 0.0804 - binary_accuracy: 0.9792 - val_loss: 0.1644 - val_binary_accuracy: 0.9444
Epoch 257/500
144/144 [==============================] - 0s 297us/step - loss: 0.0886 - binary_accuracy: 0.9722 - val_loss: 0.1641 - val_binary_accuracy: 0.9444
Epoch 258/500
144/144 [==============================] - 0s 320us/step - loss: 0.0765 - binary_accuracy: 0.9861 - val_loss: 0.1644 - val_binary_accuracy: 0.9444
Epoch 259/500
144/144 [==============================] - 0s 327us/step - loss: 0.0702 - binary_accuracy: 0.9792 - val_loss: 0.1641 - val_binary_accuracy: 0.9444
Epoch 260/500
144/144 [==============================] - 0s 307us/step - loss: 0.0961 - binary_accuracy: 0.9653 - val_loss: 0.1640 - val_binary_accuracy: 0.9444
Epoch 261/500
144/144 [==============================] - 0s 322us/step - loss: 0.0780 - binary_accuracy: 0.9722 - val_loss: 0.1641 - val_binary_accuracy: 0.9444
Epoch 262/500
144/144 [===========

Epoch 307/500
144/144 [==============================] - 0s 287us/step - loss: 0.0636 - binary_accuracy: 0.9861 - val_loss: 0.1644 - val_binary_accuracy: 0.9444
Epoch 308/500
144/144 [==============================] - 0s 272us/step - loss: 0.0802 - binary_accuracy: 0.9722 - val_loss: 0.1642 - val_binary_accuracy: 0.9444
Epoch 309/500
144/144 [==============================] - 0s 270us/step - loss: 0.0708 - binary_accuracy: 0.9861 - val_loss: 0.1639 - val_binary_accuracy: 0.9444
Epoch 310/500
144/144 [==============================] - 0s 280us/step - loss: 0.0954 - binary_accuracy: 0.9792 - val_loss: 0.1639 - val_binary_accuracy: 0.9444
Epoch 311/500
144/144 [==============================] - 0s 313us/step - loss: 0.0901 - binary_accuracy: 0.9653 - val_loss: 0.1637 - val_binary_accuracy: 0.9444
Epoch 312/500
144/144 [==============================] - 0s 307us/step - loss: 0.0785 - binary_accuracy: 0.9861 - val_loss: 0.1641 - val_binary_accuracy: 0.9444
Epoch 313/500
144/144 [===========

Epoch 358/500
144/144 [==============================] - 0s 305us/step - loss: 0.0820 - binary_accuracy: 0.9792 - val_loss: 0.1637 - val_binary_accuracy: 0.9444
Epoch 359/500
144/144 [==============================] - 0s 323us/step - loss: 0.0573 - binary_accuracy: 0.9931 - val_loss: 0.1643 - val_binary_accuracy: 0.9444
Epoch 360/500
144/144 [==============================] - 0s 280us/step - loss: 0.0704 - binary_accuracy: 0.9792 - val_loss: 0.1641 - val_binary_accuracy: 0.9444
Epoch 361/500
144/144 [==============================] - 0s 330us/step - loss: 0.0663 - binary_accuracy: 0.9931 - val_loss: 0.1640 - val_binary_accuracy: 0.9444
Epoch 362/500
144/144 [==============================] - 0s 287us/step - loss: 0.0706 - binary_accuracy: 0.9722 - val_loss: 0.1644 - val_binary_accuracy: 0.9444
Epoch 363/500
144/144 [==============================] - 0s 286us/step - loss: 0.1017 - binary_accuracy: 0.9792 - val_loss: 0.1640 - val_binary_accuracy: 0.9444
Epoch 364/500
144/144 [===========

Epoch 409/500
144/144 [==============================] - 0s 301us/step - loss: 0.0704 - binary_accuracy: 0.9792 - val_loss: 0.1629 - val_binary_accuracy: 0.9444
Epoch 410/500
144/144 [==============================] - 0s 314us/step - loss: 0.0829 - binary_accuracy: 0.9722 - val_loss: 0.1626 - val_binary_accuracy: 0.9444
Epoch 411/500
144/144 [==============================] - 0s 472us/step - loss: 0.0605 - binary_accuracy: 0.9792 - val_loss: 0.1623 - val_binary_accuracy: 0.9444
Epoch 412/500
144/144 [==============================] - 0s 350us/step - loss: 0.1322 - binary_accuracy: 0.9444 - val_loss: 0.1625 - val_binary_accuracy: 0.9444
Epoch 413/500
144/144 [==============================] - 0s 348us/step - loss: 0.0927 - binary_accuracy: 0.9861 - val_loss: 0.1627 - val_binary_accuracy: 0.9444
Epoch 414/500
144/144 [==============================] - 0s 349us/step - loss: 0.0753 - binary_accuracy: 0.9861 - val_loss: 0.1629 - val_binary_accuracy: 0.9444
Epoch 415/500
144/144 [===========

Epoch 460/500
144/144 [==============================] - 0s 383us/step - loss: 0.0739 - binary_accuracy: 0.9861 - val_loss: 0.1628 - val_binary_accuracy: 0.9444
Epoch 461/500
144/144 [==============================] - 0s 389us/step - loss: 0.0691 - binary_accuracy: 0.9792 - val_loss: 0.1626 - val_binary_accuracy: 0.9444
Epoch 462/500
144/144 [==============================] - 0s 285us/step - loss: 0.1049 - binary_accuracy: 0.9653 - val_loss: 0.1623 - val_binary_accuracy: 0.9444
Epoch 463/500
144/144 [==============================] - 0s 320us/step - loss: 0.1111 - binary_accuracy: 0.9514 - val_loss: 0.1624 - val_binary_accuracy: 0.9444
Epoch 464/500
144/144 [==============================] - 0s 323us/step - loss: 0.0643 - binary_accuracy: 0.9931 - val_loss: 0.1622 - val_binary_accuracy: 0.9444
Epoch 465/500
144/144 [==============================] - 0s 290us/step - loss: 0.0701 - binary_accuracy: 1.0000 - val_loss: 0.1620 - val_binary_accuracy: 0.9444
Epoch 466/500
144/144 [===========

## Testing

### Preprocess Test Set

In [9]:
test = pd.read_csv('dataset/test_values.csv')

# Convert string values to integers
test["thal"] = test["thal"].astype('category')
test["thal"] = test["thal"].cat.codes

# Whiten test set
test[cols] = (test[cols] - train_mean) / train_std

display(test.head())

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,olalu7,0.788226,1.184944,2.327660,-2.232814,-0.752141,-0.41176,0.993080,0.835944,-0.745370,0.715687,0.443077,0.431837,-0.693665
1,z9n6mx,-0.905871,-0.611182,0.420163,0.898925,-0.752141,-0.41176,-1.006969,-1.355365,0.403604,-1.387556,-2.121869,1.461531,-0.693665
2,5k4413,0.788226,1.184944,-0.652804,0.898925,-0.752141,-0.41176,0.993080,-1.480583,1.456830,0.715687,-1.266887,-1.314166,1.431607
3,mrg7q5,-0.905871,-0.611182,-1.725771,-0.144988,0.299396,-0.41176,-1.006969,1.462032,-0.936866,-1.387556,0.549949,0.476607,-0.693665
4,uki4do,0.788226,-0.611182,0.420163,0.898925,0.299396,-0.41176,0.993080,-1.710149,2.510056,0.715687,0.656822,-1.090319,1.431607


In [10]:
predictions = model.predict(test[cols].values)

output = pd.DataFrame({
    'patient_id': test['patient_id'],
    'heart_disease_present': predictions[:,0]
})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(output)

output_dir = 'output'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
output_file = output_dir + '/' + experiment_name + '-' + 'out.csv'
output.to_csv(output_file, index=False)

,patient_id,heart_disease_present
0,olalu7,0.938102
1,z9n6mx,0.002144
2,5k4413,0.986931
3,mrg7q5,0.043616
4,uki4do,0.969927
5,kev1sk,0.007098
6,9n6let,0.029613
7,jxmtyg,0.999568
8,51s2ff,0.411496
9,wi9mcs,0.216490
